In [225]:
import pandas as pd 
import dask.dataframe as dd
import pyarrow as pa
import numpy as np
import tqdm
from datetime import datetime as dt
from tsfresh.feature_extraction import extract_features
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from scipy import stats
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot
import scipy
import seaborn
import jieba
# jieba.enable_parallel(4)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import lightgbm as lgbm

In [226]:
def binaryRating(rating):
    return 1 if rating > 3 else 0

In [227]:
collection = MongoClient('10.160.1.15:27017')['googleplay']['comments']

In [228]:
df = pd.DataFrame(list(collection.find({},{'comments':1,'ratings':1})))

In [230]:
df.to_csv("google_play.csv")

In [197]:
# df = df[['comments','ratings']]

In [199]:
wordCount = df.apply(lambda x:len(str(x['comments']).strip()),axis=1)

In [200]:
df = df[(wordCount>=4) & (df['ratings']!='3')]

In [201]:
cut = df[['comments']].apply(lambda x: " ".join(jieba.cut(x['comments'].strip() + " $ENDING$",cut_all=False)),axis=1)
# cut = df[['comments']].apply(lambda x: " ".join(x['comments']) + " $ENDING$",axis=1)

In [202]:
from gensim.corpora import WikiCorpus

In [203]:
# wiki_corpus = WikiCorpus('zhwiki-20190320-pages-articles-multistream.xml.bz2', dictionary={})

In [204]:
texts_num = 0
with open("wiki_zh_tw.txt",'w',encoding='utf-8') as output:
    for text in df['comments']:
        output.write(text + '\n')
        texts_num += 1
        if texts_num % 1000 == 0:
            print("已處理 %d 篇文章" % texts_num)
    output.close

已處理 1000 篇文章
已處理 2000 篇文章
已處理 3000 篇文章
已處理 4000 篇文章
已處理 5000 篇文章
已處理 6000 篇文章
已處理 7000 篇文章
已處理 8000 篇文章
已處理 9000 篇文章
已處理 10000 篇文章
已處理 11000 篇文章
已處理 12000 篇文章
已處理 13000 篇文章
已處理 14000 篇文章
已處理 15000 篇文章
已處理 16000 篇文章
已處理 17000 篇文章
已處理 18000 篇文章
已處理 19000 篇文章
已處理 20000 篇文章
已處理 21000 篇文章
已處理 22000 篇文章
已處理 23000 篇文章
已處理 24000 篇文章
已處理 25000 篇文章
已處理 26000 篇文章
已處理 27000 篇文章
已處理 28000 篇文章
已處理 29000 篇文章
已處理 30000 篇文章
已處理 31000 篇文章
已處理 32000 篇文章
已處理 33000 篇文章
已處理 34000 篇文章
已處理 35000 篇文章
已處理 36000 篇文章
已處理 37000 篇文章
已處理 38000 篇文章
已處理 39000 篇文章
已處理 40000 篇文章
已處理 41000 篇文章
已處理 42000 篇文章
已處理 43000 篇文章
已處理 44000 篇文章
已處理 45000 篇文章
已處理 46000 篇文章
已處理 47000 篇文章
已處理 48000 篇文章
已處理 49000 篇文章
已處理 50000 篇文章
已處理 51000 篇文章
已處理 52000 篇文章
已處理 53000 篇文章
已處理 54000 篇文章
已處理 55000 篇文章
已處理 56000 篇文章
已處理 57000 篇文章
已處理 58000 篇文章
已處理 59000 篇文章
已處理 60000 篇文章
已處理 61000 篇文章
已處理 62000 篇文章
已處理 63000 篇文章
已處理 64000 篇文章
已處理 65000 篇文章
已處理 66000 篇文章
已處理 67000 篇文章
已處理 68000 篇文章
已處理 69000 篇文章
已處理 70000 篇文章
已處理 71000 篇文章
已處理 72000 篇文章
已

In [205]:
output = open('wiki_seg.txt', 'w', encoding='utf-8')
with open('wiki_zh_tw.txt', 'r', encoding='utf-8') as content :
    for texts_num, line in enumerate(content):
        line = line.strip('\n')
        words = jieba.cut(line, cut_all=False)
        for word in words:
#             if word not in stopword_set:
            output.write(word + ' ')
        output.write('\n')

        if (texts_num + 1) % 10000 == 0:
            print("已完成前 %d 行的斷詞" % (texts_num + 1))

已完成前 10000 行的斷詞
已完成前 20000 行的斷詞
已完成前 30000 行的斷詞
已完成前 40000 行的斷詞
已完成前 50000 行的斷詞
已完成前 60000 行的斷詞
已完成前 70000 行的斷詞
已完成前 80000 行的斷詞
已完成前 90000 行的斷詞
已完成前 100000 行的斷詞
已完成前 110000 行的斷詞
已完成前 120000 行的斷詞
已完成前 130000 行的斷詞
已完成前 140000 行的斷詞
已完成前 150000 行的斷詞
已完成前 160000 行的斷詞
已完成前 170000 行的斷詞
已完成前 180000 行的斷詞
已完成前 190000 行的斷詞
已完成前 200000 行的斷詞
已完成前 210000 行的斷詞
已完成前 220000 行的斷詞
已完成前 230000 行的斷詞
已完成前 240000 行的斷詞


In [206]:
MAX_SENTENCE_LENGTH = 50
WORD_VECTOR_DIM = 25

In [207]:
from gensim.test.utils import common_texts
from gensim.sklearn_api import W2VTransformer
from gensim import models
from gensim.models import word2vec
from gensim.models import Word2Vec
# Create a model to represent each word by a 10 dimensional vector.
sentences = models.word2vec.Text8Corpus(cut[0])

sentences = word2vec.LineSentence("wiki_seg.txt")
wvmodel = word2vec.Word2Vec(sentences, size=250)
wvmodel = Word2Vec(sentences, size=WORD_VECTOR_DIM, window=5, min_count=0, workers=3)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [208]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Activation, Flatten, Dropout, GRU, BatchNormalization
from keras.layers import Embedding

In [209]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

tokenizer = Tokenizer(nb_words=1000000000)
tokenizer.fit_on_texts(cut)
sequences = tokenizer.texts_to_sequences(cut)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/home/daniel/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:175: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 135230 unique tokens.


In [210]:
data = pad_sequences(sequences, maxlen=MAX_SENTENCE_LENGTH)

In [211]:
embeddings_index = {}
for word in wvmodel.wv.vocab:
    coefs = np.asarray(wvmodel.wv[word], dtype='float32')
    embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 140825 word vectors.


In [212]:
embedding_matrix = np.zeros((len(word_index) + 1, WORD_VECTOR_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [213]:
embedding_layer = Embedding(len(word_index) + 1,
                            WORD_VECTOR_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SENTENCE_LENGTH,
                            trainable=False)

In [214]:
y = np.array(df['ratings'].astype(int))

In [215]:
for i in range(len(y)):
    y[i] = 1 if y[i] > 3 else 0

In [216]:
ml_model = Sequential()
ml_model.add(embedding_layer)
ml_model.add(GRU(32, return_sequences=True, activation='relu'))
# model.add(LSTM(8, return_sequences=True))
ml_model.add(Flatten())
ml_model.add(Dropout(0.5))
# model.add(Dense(32))
ml_model.add(Dense(1))

ml_model.add(Activation('sigmoid'))
ml_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [217]:
stratified_folder = StratifiedKFold(n_splits=4, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(data, y):
    X_train, X_test = data[train_index], data[test_index]
    y_train, y_test = y[train_index],y[test_index]
    ml_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, verbose=1, batch_size=256)
    pred = ml_model.predict_classes(X_test)
    print(f1_score(pred, y_test, average='macro'))
    print(confusion_matrix(pred, y_test))
    break

Train on 181635 samples, validate on 60547 samples
Epoch 1/10
181635/181635 [==============================] - 47s 258us/step - loss: 0.3672 - acc: 0.8328 - val_loss: 0.3221 - val_acc: 0.8566
Epoch 2/10
181635/181635 [==============================] - 54s 299us/step - loss: 0.3133 - acc: 0.8624 - val_loss: 0.3087 - val_acc: 0.8618
Epoch 3/10
181635/181635 [==============================] - 53s 293us/step - loss: 0.3020 - acc: 0.8674 - val_loss: 0.3013 - val_acc: 0.8696
Epoch 4/10
181635/181635 [==============================] - 54s 300us/step - loss: 0.2958 - acc: 0.8716 - val_loss: 0.2959 - val_acc: 0.8711
Epoch 5/10
181635/181635 [==============================] - 51s 282us/step - loss: 0.2919 - acc: 0.8729 - val_loss: 0.2973 - val_acc: 0.8683
Epoch 6/10
181635/181635 [==============================] - 50s 277us/step - loss: 0.2883 - acc: 0.8753 - val_loss: 0.2950 - val_acc: 0.8711
Epoch 7/10
181635/181635 [==============================] - 60s 330us/step - loss: 0.2852 - acc: 0.8763

In [218]:
import pickle
ml_model.save("api_test.h5")
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [219]:
test = ['沒有一次登入成功的 可以爛到這樣也是很猛']

In [220]:
test = list(map(lambda x : " ".join(jieba.cut(x.strip() + " $ENDING$")), test))

In [221]:
test_sequences = tokenizer.texts_to_sequences(test)

In [222]:
data = pad_sequences(sequences, maxlen=MAX_SENTENCE_LENGTH)

In [224]:
ml_model.predict_proba(data)

KeyboardInterrupt: 

In [232]:
import requests

<Response [201]>

In [ ]:
ml_model.predict_proba(X_test)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix,f1_score
from sklearn.naive_bayes import MultinomialNB, ComplementNB
import lightgbm as lgbm

vectorizer = CountVectorizer(max_df=0.3, min_df=0).fit(cut)
counts_vector = vectorizer.transform(cut)

tfidfVectorizer = TfidfTransformer(smooth_idf=True).fit(counts_vector)
tfidf_vector = tfidfVectorizer.transform(counts_vector)

In [62]:
stratified_folder = StratifiedKFold(n_splits=4, random_state=0, shuffle=False) 
for train_index, test_index in stratified_folder.split(tfidf_vector, y): 
    X_train, X_test = tfidf_vector[train_index], tfidf_vector[test_index]
    y_train, y_test = y[train_index], y[test_index] 
    
    lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_classes':1,
        'subsample': .9,
        'colsample_bytree': .5,
        'reg_alpha': .1,
        'reg_lambda': .1,
        'min_split_gain': 0.001,
        'min_child_weight': 5,
        'silent':True,
        'verbosity':-1,
        'nthread':2
    }

    clf = lgbm.LGBMClassifier(**lgb_params,learning_rate=0.05,
                                    n_estimators=10000,max_depth=4)
    clf.fit(
        X_train, y_train,
        eval_set=[(X_train, y_train),(X_test, y_test)],
        verbose=50,
        early_stopping_rounds=50
    )
    pred = clf.predict(X_test, num_iteration=clf.best_iteration_)
    print(f1_score(pred, y_test, average='macro'))
    print(confusion_matrix(pred, y_test))

Training until validation scores don't improve for 50 rounds.
[50]	valid_0's binary_logloss: 0.480021	valid_1's binary_logloss: 0.485176
[100]	valid_0's binary_logloss: 0.447456	valid_1's binary_logloss: 0.455579
[150]	valid_0's binary_logloss: 0.428476	valid_1's binary_logloss: 0.438524
[200]	valid_0's binary_logloss: 0.415527	valid_1's binary_logloss: 0.426968
[250]	valid_0's binary_logloss: 0.40567	valid_1's binary_logloss: 0.418001
[300]	valid_0's binary_logloss: 0.397815	valid_1's binary_logloss: 0.411123
[350]	valid_0's binary_logloss: 0.391154	valid_1's binary_logloss: 0.405144
[400]	valid_0's binary_logloss: 0.385503	valid_1's binary_logloss: 0.40022
[450]	valid_0's binary_logloss: 0.380543	valid_1's binary_logloss: 0.395811
[500]	valid_0's binary_logloss: 0.376171	valid_1's binary_logloss: 0.392053
[550]	valid_0's binary_logloss: 0.372194	valid_1's binary_logloss: 0.388535
[600]	valid_0's binary_logloss: 0.368598	valid_1's binary_logloss: 0.385468
[650]	valid_0's binary_loglos

KeyboardInterrupt: 

In [ ]:
clf.predict_proba(X_test)

In [ ]:
test = '爛'
test = " ".join(jieba.cut(test))
test = tfidfVectorizer.transform(vectorizer.transform([test]))
model.predict_proba(test)